In [1]:
import hickle
from ase.io import read
import pyscf.pbc.gto as pbcgto
import pyscf.pbc.tools.pyscf_ase as pyscf_ase
from pyscf.pbc.tools.k2gamma import get_phase, kpts_to_kmesh, k2gamma
import numpy as np
import pyscf.pbc.scf as pbcscf
from ase.visualize import view

/home/nigam/miniconda3/lib/python3.11/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [2]:
from ase.atoms import Atoms
frame= Atoms('C', positions = [[0,0,0.1]], cell=[2,2,1])
frame.center()
frame22 = frame.repeat([2,2,1])
#view([frame, frame22])

In [45]:
view([frame, frame22])

<Popen: returncode: None args: ['/home/nigam/miniconda3/bin/python', '-m', '...>

In [16]:
# frame = read('examples/data/periodic/graphene_supercell_variety/graphene_variety.xyz', '1')


In [53]:
kmesh = [2,2,1]
cell = pbcgto.Cell()
cell.atom = pyscf_ase.ase_atoms_to_pyscf(frame)

cell.basis = 'sto-3g' #'def2-svp'
cell.a = frame.cell
cell.verbose = 3
print('-------------------------------------------------------------------------------', flush= True)
cell.build()

kmesh = kmesh
print('KMESH', kmesh)
kpts = cell.make_kpts(kmesh)
#kmesh = [np.int32(np.max([1, np.ceil(2*np.pi*np.linalg.norm(vec)/kspacing)])) for vec in frame.cell.reciprocal().array]

nao = cell.nao
kmf = pbcscf.KRHF(cell, kpts = cell.make_kpts(kmesh))

kmf = kmf.density_fit()
kmf.init_guess_by_minao() #('minao')
kmf.max_cycle = 500
kmf.conv_tol = 1e-8 
kmf.conv_tol_grad = 1e-8

# kmf.kernel()

-------------------------------------------------------------------------------
KMESH [2, 2, 1]


In [47]:
fock = kmf.get_fock(dm =kmf.init_guess_by_minao(), cycle= 0 )

In [18]:
# hex22 = read('examples/data/periodic/graphene_supercell_variety/graphene_variety.xyz', '2')

kmesh = [1,1,1]

In [67]:
supercell = pbcgto.Cell()
supercell.atom = pyscf_ase.ase_atoms_to_pyscf(frame22)

supercell.basis = 'sto-3g' #'def2-svp'
supercell.a = frame.cell
supercell.verbose = 3
print('-------------------------------------------------------------------------------', flush= True)
supercell.build()

kmesh = kmesh
print('KMESH', kmesh)
kpts = supercell.make_kpts(kmesh)
#kmesh = [np.int32(np.max([1, np.ceil(2*np.pi*np.linalg.norm(vec)/kspacing)])) for vec in frame.cell.reciprocal().array]

nao = cell.nao
superkmf = pbcscf.KRHF(supercell, kpts = kpts)

superkmf = superkmf.density_fit()
superkmf.max_cycle = 0
superkmf.conv_tol = 1e-8 
superkmf.conv_tol_grad = 1e-8

superkmf.kernel()

-------------------------------------------------------------------------------
KMESH [2, 2, 1]

WARN: Singularity detected in overlap matrix.  Integral accuracy may be not enough.
      You can adjust  cell.precision  or  cell.rcut  to improve accuracy.  Recommended settings are
      cell.precision < 3.2e-13
      cell.rcut > 21.38


WARN: Singularity detected in overlap matrix.  Integral accuracy may be not enough.
      You can adjust  cell.precision  or  cell.rcut  to improve accuracy.  Recommended settings are
      cell.precision < 3.2e-13
      cell.rcut > 21.38


WARN: Singularity detected in overlap matrix (condition number = 3.1e+19). SCF may be inaccurate and hard to converge.



LinAlgError: The leading minor of order 6 of B is not positive definite. The factorization of B could not be completed and no eigenvalues or eigenvectors were computed.

In [39]:
fock22 = superkmf.get_fock(dm = superkmf.init_guess_by_minao(), cycle= 0 )


WARN: Singularity detected in overlap matrix.  Integral accuracy may be not enough.
      You can adjust  cell.precision  or  cell.rcut  to improve accuracy.  Recommended settings are
      cell.precision < 8.8e-12
      cell.rcut > 20.64



In [66]:
kmf.max_cycle

500

In [40]:
fock 

NameError: name 'fock' is not defined

In [ ]:
superfock

In [56]:
over = kmf.get_ovlp()

In [64]:
over[0][:5,:5] - over22[0][:5,:5]

array([[-2.22044605e-16+0.j,  2.22044605e-16+0.j, -7.63430795e-18+0.j,
        -2.17298294e-18+0.j, -7.38129657e-18+0.j],
       [ 2.22044605e-16+0.j,  1.77635684e-15+0.j, -1.35891588e-17+0.j,
        -6.40041057e-17+0.j,  3.51437940e-18+0.j],
       [-6.67631369e-18+0.j,  5.52225222e-17+0.j, -6.66133815e-16+0.j,
         2.69335301e-18+0.j, -6.91524316e-18+0.j],
       [-5.61095337e-19+0.j, -4.06840843e-17+0.j,  6.16279997e-18+0.j,
         6.66133815e-16+0.j, -4.39550013e-17+0.j],
       [ 9.85174054e-18+0.j, -1.46889211e-17+0.j, -2.08129362e-17+0.j,
        -2.31247671e-17+0.j,  1.15185639e-15+0.j]])

In [61]:
over22[0]

array([[ 1.00075836e+00,  5.12886055e-01,  5.66046828e-18,
         1.62121103e-18, -1.30105521e-18,  1.00075836e+00,
         5.12886055e-01,  5.79632959e-18,  2.02099173e-18,
        -3.02763467e-17,  1.00075836e+00,  5.12886055e-01,
         7.70425760e-18, -1.73288908e-19, -2.76740752e-17,
         1.00075836e+00,  5.12886055e-01,  7.55823573e-18,
        -2.70481803e-19, -2.66245534e-17],
       [ 5.12886055e-01,  4.25527279e+00, -1.88728049e-16,
        -1.09489265e-17, -4.78089483e-17,  5.12886055e-01,
         4.25527279e+00, -3.16176037e-16,  4.80896736e-17,
        -7.60937653e-17,  5.12886055e-01,  4.25527279e+00,
        -1.86913220e-16,  3.37880647e-17, -7.73443622e-17,
         5.12886055e-01,  4.25527279e+00, -2.69904409e-13,
        -2.69735177e-13, -7.60443705e-17],
       [ 1.71380056e-18,  1.47094685e-16,  1.01591954e+00,
        -6.04987989e-18,  3.27410130e-17,  1.13599981e-18,
         2.03419011e-16,  1.01591954e+00, -2.16422758e-17,
         9.61940515e-18, -7.7

In [59]:
over22 = superkmf.get_ovlp()


WARN: Singularity detected in overlap matrix.  Integral accuracy may be not enough.
      You can adjust  cell.precision  or  cell.rcut  to improve accuracy.  Recommended settings are
      cell.precision < 8.8e-12
      cell.rcut > 20.64



In [52]:
over -over22[:5,:5]

array([[ 0.00000000e+00,  1.11022302e-16, -7.63346092e-18,
        -2.87438354e-18,  9.90234455e-19],
       [ 1.11022302e-16,  0.00000000e+00,  2.12792334e-16,
         3.31573496e-17, -1.32647808e-17],
       [ 7.19978005e-18, -1.72893694e-16,  1.77635684e-15,
         6.66760513e-18, -6.91524316e-18],
       [-8.12008020e-19, -1.05909016e-16,  6.69471019e-18,
         1.11022302e-15, -4.39550013e-17],
       [ 9.85174054e-18, -1.46889211e-17, -3.94121783e-17,
         8.74827539e-18,  1.15185639e-15]])

In [30]:
fock = kmf.get_fock()
fock22 = superkmf.get_fock()

TypeError: object of type 'NoneType' has no len()